In [1]:
import nltk
from nltk.util import bigrams
from collections import Counter

In [2]:
#  a string from the BeRP dataset
text = './transcript.txt' # Path to the BeRP dataset

In [3]:
# Tokenize the text
tokens = nltk.word_tokenize(text.lower())  # Lowercase to maintain consistency

In [4]:
# Generate bigrams
bi_grams = list(bigrams(tokens))

In [5]:
# Calculate frequency distributions for unigrams and bigrams
unigram_freq = Counter(tokens)
bigram_freq = Counter(bi_grams)

In [6]:
# Calculate bigram probabilities
bigram_probabilities = {}
for bigram, count in bigram_freq.items():
    w1 = bigram[0]
    w2 = bigram[1]
    bigram_probabilities[(w1, w2)] = count / unigram_freq[w1]

In [7]:
# Function to calculate sentence probability
def calculate_sentence_probability(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    prob = 1.0
    for i in range(1, len(tokens)):
        w1 = tokens[i - 1]
        w2 = tokens[i]
        prob *= bigram_probabilities.get((w1, w2), 0)  # If bigram doesn't exist, set probability to 0
    return prob

In [8]:
# Example sentences
sentence_1 = "show me all the Arabic food restaurants"
sentence_2 = "I am learning mathematics"

In [9]:
# Calculate sentence probabilities
prob_1 = calculate_sentence_probability(sentence_1)
prob_2 = calculate_sentence_probability(sentence_2)

print(f"Probability of Sentence 1: {prob_1}")
print(f"Probability of Sentence 2: {prob_2}")

Probability of Sentence 1: 0.0
Probability of Sentence 2: 0.0
